In [ ]:
import logging
import math
import pandas as pd
import numpy as np
from scipy.interpolate import UnivariateSpline
from matplotlib import pyplot as plt
from scipy.stats import rankdata
from sklearn.preprocessing import normalize
from math import log
from mpl_toolkits.axes_grid1 import make_axes_locatable   
import random
from mpl_toolkits.mplot3d import axes3d

from scipy.special import logit as logit


class Logger(object):
    def __init__(self, log_file_name, log_level, logger_name):
        self.__logger = logging.getLogger(logger_name)
        self.__logger.setLevel(log_level)
        file_handler = logging.FileHandler(log_file_name)
        console_handler = logging.StreamHandler()
        formatter = logging.Formatter(
            '[%(asctime)s] - [%(filename)s line:%(lineno)d] : %(message)s')
        file_handler.setFormatter(formatter)
        console_handler.setFormatter(formatter)
        self.__logger.addHandler(file_handler)
        self.__logger.addHandler(console_handler)

    def get_log(self):
        return self.__logger


def scale(X, x_min, x_max):
    nom = (X-X.min(axis=0))*(x_max-x_min)
    denom = X.max(axis=0) - X.min(axis=0)
    denom[denom==0] = 1
    return x_min + nom/denom
    
    
logger = Logger('C:\\Users\\bird\\Desktop\\cancer_project_github\\log.txt', log_level=logging.DEBUG, logger_name="CIFAR").get_log()

clinical_231_file = 'C:\\Users\\bird\Desktop\\cancer_project_github\\cancer_data\\clinical_231_data.csv'
mRNA_231_file = 'C:\\Users\\bird\\Desktop\\cancer_project_github\\cancer_data\\data_mRNA_231_sort_with_sensitive.csv'




In [105]:
"""
load data
"""
filterd_data = 'C:\\Users\\bird\Desktop\\cancer_project_github\\cancer_data\\filtered_187_genes_data.csv'
mRNA_231_df=pd.read_csv(filterd_data)

st = [str(x) for x in range(231)]

mRNA_231_df.drop_duplicates(subset=st)

mRNA_231_df = mRNA_231_df.set_index('SAMPLE_ID')
print(mRNA_231_df)
print(mRNA_231_df)


               0       1       2       3       4          5       6        7  \
SAMPLE_ID                                                                      
2          8.693  22.395  50.843  66.151  54.679  11564.212  43.055  394.181   
9          1.164  32.202  58.076  29.167  52.251   8333.306  18.559  101.457   
12         0.000   9.185   0.000   0.000   0.574      0.198   0.752    1.472   
15         0.000   0.000   0.000   0.000   0.418      0.313   0.402    1.026   
18         0.000   3.939   0.000   0.000   0.425      0.186   0.263    0.791   
...          ...     ...     ...     ...     ...        ...     ...      ...   
13683      0.027   0.053   0.028   0.134   0.243      0.000   0.188    0.123   
13749      0.000   0.006   0.007   0.000   0.009      0.014   0.037    0.000   
13889      0.024   0.000   0.000   0.854   0.000      0.268   0.000    0.000   
13940      0.016   0.044   0.032   1.883   0.007      0.000   0.000    0.022   
14119      0.006   0.021   0.025   0.011

In [110]:
mRNA_231_df=pd.read_csv(filterd_data)
mRNA_231_df = mRNA_231_df.set_index('SAMPLE_ID')
patient_mRNA = []
for column in mRNA_231_df:
    #     print(mRNA_244_df[column].tolist())
    patient_mRNA.append(mRNA_231_df[column].tolist())
mRNA_np = np.asarray(patient_mRNA)



print(mRNA_np.shape)
    


sorts = {"":'0',
    "homo":'1',
    "hetero":"2"}

norms = {'': '0',
         'linear_scale_and_bin': 'A',
         'log_scale_and_bin': 'A',
         'rank_and_scale': 'B'
}

params = ['homo','mean', 'rank_and_scale', sorts, norms] # scheme = {'', 'hetero' 'homo'}, norm = {'','rank_and_scale', 'scale_and_bin_linear', 'log_scale_and_bin'}
norm = 'rank_and_scale'
# patient_np = plot_genes[i].T
#     patient_np = plot_genes[i,3].T   # uncomment this for intersect_genes only

correlation_matrix, patient_np = norm_and_sort_patients(mRNA_np, params)#hetero homo scale_and_bin rank_and_scale


# plot_heatmap(correlation_matrix,params)


(231, 187)


# ignore below !#

In [111]:
"""
load data
"""
mRNA_231_df=pd.read_csv(mRNA_231_file)

st = [str(x) for x in range(231)]

mRNA_231_df.drop_duplicates(subset=st)

mRNA_231_df = mRNA_231_df.set_index('SAMPLE_ID')
print(mRNA_231_df)
print(mRNA_231_df)


                   0          1          2          3          4           5  \
SAMPLE_ID                                                                      
0          12880.736  39235.245  47125.027  17759.562  16911.939  373520.889   
1          13629.979  29296.085  42686.075  14363.479  12011.571  157101.452   
2              8.693     22.395     50.843     66.151     54.679   11564.212   
3           1926.368   9021.929  12278.160   8388.242  12418.195   74725.583   
4           8592.185  19165.452  13258.599   9400.131   9190.623  150989.122   
...              ...        ...        ...        ...        ...         ...   
14374          0.000      0.000      0.000      0.000      0.000       0.000   
14375          0.000      0.000      0.000      0.000      0.000       0.000   
14376          0.000      0.000      0.000      0.000      0.000       0.000   
14377          0.000      0.000      0.000      0.000      0.000       0.000   
14378          0.000      0.000      0.0

In [45]:

def norm_and_sort_patients(patient, params ):
    scheme, sort_standard,  norm, sort_s, norm_s= params
    patient_np = np.copy(patient)
    if norm == 'linear_scale_and_bin':
        patient_np = np.floor(scale(patient_np, 0, 100))
    if norm == 'log_scale_and_bin':
        patient_np = np.log10(patient_np+0.0000001)
        patient_np = np.floor(scale(patient_np, 0, 100))
    if norm == 'rank_and_scale': 
        x = patient_np.T
        def rd(x):
            return  (rankdata(x, method='dense') - 1).astype(int)
        x = np.apply_along_axis(rd, 1, x)
        patient_np = scale(x.T,0,100)
        
      
    r = np.corrcoef(patient_np, patient_np)
    ranks = []
    
        
    if scheme == '':
        return r[0:231, 0:231], patient_np
    
    if scheme == 'homo':
#         print('homo')
#         corr_means = sort_standards(sort_standard, r[:231,:231])
#         print(corr_means.shape)
        correlation_matrix = r[0:117, 0:117] #LN 0-117, Bone 117-191, Liver 191-231
        corr_means = sort_standards(sort_standard, correlation_matrix)
#         print(corr_means)
        ranks = np.argsort(-corr_means)
        correlation_matrix = r[117:191, 117:191] #LN 0-117, Bone 117-191, Liver 191-231
        corr_means = sort_standards(sort_standard, correlation_matrix)
        ranks_t= np.argsort(-corr_means) + 117
        ranks = np.concatenate([ranks, ranks_t]) 
        correlation_matrix = r[191:231, 191:231] #LN 0-117, Bone 117-191, Liver 191-231
        corr_means = sort_standards(sort_standard, correlation_matrix)
        ranks_t = np.argsort(-corr_means) + 191
        ranks = np.concatenate([ranks, ranks_t]) 

    if scheme == 'hetero':
        correlation_matrix = r[ 0:231,0:117] #LN 0-117, Bone 117-191, Liver 191-231
        corr_means = sort_standards(sort_standard, correlation_matrix)
        ranks = np.argsort(-corr_means)
        correlation_matrix = r[0:231, 117:191] #LN 0-117, Bone 117-191, Liver 191-231
        corr_means = sort_standards(sort_standard, correlation_matrix)
        ranks_t= np.argsort(-corr_means) + 117
        ranks = np.concatenate([ranks, ranks_t]) 
        correlation_matrix = r[0:231, 191:231] #LN 0-117, Bone 117-191, Liver 191-231
        corr_means = sort_standards(sort_standard, correlation_matrix)
        ranks_t = np.argsort(-corr_means) + 191
        ranks = np.concatenate([ranks, ranks_t]) 

    correlation_matrix = r[0:231, 0:231]
    correlation_matrix = correlation_matrix[:, ranks][ranks]
    
#     correlation_matrix = np.around(correlation_matrix, decimals=2)
    
    return correlation_matrix, patient_np


def plot_heatmap(correlation_matrix, params):
#     r = np.corrcoef(patient_np, patient_np)

#     correlation_matrix = r[0:231, 0:231]
    # correlation_matrix = scale_floor(correlation_matrix, 0, 100) # normalisation method 1

#     print(correlation_matrix.mean(axis=0))
    scheme, sort_standad, norm, sorts, norms= params
    if not scheme in sorts:
        scheme = ''
    if not norm in norms:
        norm = ''
    ref_1 = norms.get(norm)
    ref_2 = sorts.get(scheme)
    if scheme: 
        scheme = '(Sort: '+scheme
        if norm:
            norm = ', Norm: ' + norm +')'
        else:
            norm = ', Norm: None)'
    else:
        scheme = '(Sort: None'
        if norm:
            norm = ', Norm: ' + norm +')'
        else:
            norm = ', Norm: None)'
        
    f = plt.figure(figsize=(20,20))
    
    plt.rc('xtick',labelsize=20)
    plt.rc('ytick',labelsize=20)
    
    ax = f.add_subplot(111)
    ax.yaxis.tick_right()

    # LN 117, Bone 74, Liver 40,Prostate 7,Lung 6
    plt.plot([117, 117], [0, 231], 'yellow')
    plt.plot([0, 244], [117, 117], 'yellow')
    plt.plot([191, 191], [0, 231], 'yellow')
    plt.plot([0, 244], [191, 191], 'yellow')
    plt.text(111, -1.5, 'LN-Bone', fontsize=28)
    plt.text(-6, 126, 'Bone-LN', rotation = 90, fontsize=28)
    plt.text(180, -1.5, 'Bone-Liver', fontsize=28)
    plt.text(-6,200, 'Liver-Bone', rotation = 90, fontsize=28)
    img = plt.imshow(correlation_matrix, cmap='seismic',vmin=-1, vmax=1, interpolation='nearest') #https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
#     title = ref_1+'-'+ ref_2+' Heatmap ' + scheme + norm
#     plt.title(title, y=-0.07, fontsize=32)
#     ax = plt.subplot(111)
#     divider = make_axes_locatable(ax)
#     cax = divider.append_axes("right", size="2%", pad=1)

#     plt.colorbar(img,cax=cax)
    plt.show()
   
    
    

def sort_standards(sort_std, corrs):
    if sort_std == 'mean':
        return np.mean(corrs, axis=0)
    elif sort_std == 'median':
        corrs = np.median(corrs, axis=0)
        return corrs
    elif sort_std == 'max':
        twmp = np.copy(corrs)
        twmp[twmp>0.999]=0
        return twmp.max(axis=0)
    elif sort_std == 'min':
        return corrs.min(axis=0)
    elif sort_std == 'weight':
        twmp = np.copy(corrs)
        twmp[twmp>0.999]=0
        twmp = list(twmp.max(axis=0))
        mini1 = list(corrs.min(axis=0))
        mean1 = list(np.mean(corrs, axis=0))
        median1 = list(np.median(corrs, axis=0))
        vrs = np.array([twmp, mini1, mean1, median1])
        vrs = np.var(vrs, axis=0)
        return vrs
    else:
        return corrs.mean(axis=0)
    

"""
given mask, threshold, norm scheme,
returns np.array(mask_corr) correlations of all genes with the mask, 
        np.array(good_mRNA) selected mRNAs shape: (n*231), 
        the_corr correlations of selected mRNAs with the mask
"""
def correlation_with_mask(mask, threshold, norm):
    (p_start, p_end) = (0,231)
    mRNA_231_df=pd.read_csv(mRNA_231_file)
    mRNA_231_df = mRNA_231_df.set_index('SAMPLE_ID')
    patient_mRNA = []
    for column in mRNA_231_df:
        #     print(mRNA_244_df[column].tolist())
        patient_mRNA.append(mRNA_231_df[column].tolist())
    mRNA_np = np.asarray(patient_mRNA)
    
    if norm == 'linear_scale_and_bin':
        mRNA_np = np.floor(scale(mRNA_np, 0, 100))
        
    if norm == 'log_scale_and_bin':
        mRNA_np = np.log10(mRNA_np+0.0000001)
        mRNA_np = np.floor(scale(mRNA_np, 0, 100))
        
    if norm == 'rank_and_scale': 
        x = mRNA_np.T
        def rd(x):
            return  (rankdata(x, method='dense') - 1).astype(int)
        x = np.apply_along_axis(rd, 1, x)
        mRNA_np = scale(x.T,0,100)
    
    if norm == 'logit_origin': #logit-normalisation on the original binned values 
        mRNA_np =scale(np.floor(scale(mRNA_np, 0, 100)), 0.00000001,0.999999999) 
        mRNA_np = logit(mRNA_np)
        
    if norm == 'logit_log': #logit-normalisation on the log binned values 
        mRNA_np = np.log10(mRNA_np+0.0000001)
        mRNA_np =scale(np.floor(scale(mRNA_np, 0, 100)), 0.00000001,0.999999999) 
        mRNA_np = logit(mRNA_np)
        
        
        
    mRNA_np = mRNA_np.T
    
    mask_corr = []  # correlations of all genes with the mask
    good_mRNA = []  # selected mRNAs shape: (n*231)
    the_corr = []   # correlations of selected mRNAs with the mask
    indexs = []
    for i in range(len(mRNA_np)):
        each = mRNA_np[i]
        corr = np.corrcoef(mask, each)[1,0]
        if corr > threshold or corr<-threshold:
#             print(i, corr)
            good_mRNA.append(each)
            the_corr.append(corr)
            indexs.append(i)
        mask_corr.append(corr)
    return np.array(mask_corr), np.array(good_mRNA), the_corr, indexs

def gene_selected_by_diff_threshold(norm,thresholds,remove_intersect= True):
    '''
    returns union_genes, intersect_genes
    '''
    LN_m = [1]*117+(231-117)*[0]
    Bone_m = [0]*117+74*[1]+(40)*[0]
    Liver_m = [0]*191+(40)*[1]
    
    LN_genes = []
    Bone_genes= []
    Liver_genes= []
    union_genes = [] # union, 3d np array (a,b,231), a is length of thresholds list, b is number of union selected genes in each threshold.
    intersect_genes = [] # intersect, 4d np array (a, 3, b, 231), a is length of thresholds list, 3 is 3 cancers intersect.
                            #b is number of union selected genes in each threshold.
    genes_remove_intersect = []
    LNBonr_indexs = []
    intersect_indexs = []
    for i in thresholds:
#         print(i)
        correlations, LNgood_mRNA, the_corr,LNindexs = correlation_with_mask(LN_m, i,norm)
        LN_genes.append(LNgood_mRNA)
        
        correlations, Bonegood_mRNA, the_corr, Boneindexs = correlation_with_mask(Bone_m, i,norm)
        Bone_genes.append(Bonegood_mRNA)
        
        correlations, Livergood_mRNA, the_corr, Liverindexs = correlation_with_mask(Liver_m, i,norm)
        Liver_genes.append(Livergood_mRNA)
        
        LNset = set([tuple(x) for x in LNgood_mRNA])
        Boneset = set([tuple(x) for x in Bonegood_mRNA])
        Liverset = set([tuple(x) for x in Livergood_mRNA])
        LN_Bone_intersect = np.array([x for x in LNset & Boneset])
        Liver_Bone_intersect = np.array([x for x in Boneset & Liverset])
        LN_Liver_intersect = np.array([x for x in Liverset & LNset])
        all_intersect = np.array([x for x in Liverset & LNset & Boneset])
        intersect_genes.append([LN_Bone_intersect, Liver_Bone_intersect, LN_Liver_intersect,all_intersect])
        
        LNBonr_indexs.append([x for x in LNindexs if x in Boneindexs]) 
        intersect_indexs.append([x for x in LNindexs if x in Boneindexs if x in Liverindexs]) 
        
        allgood_mRNA = np.concatenate((LNgood_mRNA,Bonegood_mRNA,Livergood_mRNA), axis=0) 
        allgood_mRNA = np.unique(allgood_mRNA, axis=0)
       
        union_genes.append(allgood_mRNA)
#         print(allgood_mRNA.shape)
    return np.array(union_genes), np.array(intersect_genes), none, LNBonr_indexs, intersect_indexs


In [63]:
a = [1,2,3,4]
b = [2,5,6,7,1,0]
c = [5,4,1]
[x for x in a if x in b if x in c]
print(len(LNBone_indexs[0]),len(LNBone_indexs[1]))
print(len(intersect_genes[1,0]))

2285 1151
1151


In [50]:
"""
In Table 3-1 and 3-2 for threshold 0.2 (and 0.15, respectively), 
•	for genes in both LN and Bone, we need to find those that give opposite correlations to LN and Bone filter vectors.  

"""

    
    
norm = "rank_and_scale" # linear_scale_and_bin, log_scale_and_bin, rank_and_scale, logit_origin, logit_log
LN_m = [1]*117+(231-117)*[0]
Bone_m = [0]*117+74*[1]+(40)*[0]
Liver_m = [0]*191+(40)*[1]

thresholds = [0.15,0.2]


thre = 0.15 #0.15,0.2
correlations, LNgood_mRNA, LN_corr, LNindex = correlation_with_mask(LN_m, thre,norm)
correlations, Bonegood_mRNA, Bone_corr, Boneindex = correlation_with_mask(Bone_m, thre,norm)

# LNset = set([tuple(x) for x in LNgood_mRNA])
# Boneset = set([tuple(x) for x in Bonegood_mRNA])
# LN_not_Bone = np.array([[x for x in LNset-Boneset]])  # LN_not_Bone
# LN_not_Bone = np.array([[x for x in Boneset-LNset]])  # Bone_not_LN


union_genes,intersect_genes, difference_genes, LNBone_indexs, rrr= gene_selected_by_diff_threshold(norm, thresholds, remove_intersect=True)
LN_not_Bone = []
indexs=[]
for i in range(len(intersect_genes)):
    correlations, LNgood_mRNA, LN_corr, LNindex = correlation_with_mask(LN_m, thresholds[i],norm)
    temp = intersect_genes[i,0]
    ggs = []
    idds = []
    for k in range(len(temp)):
        LNcorr = np.corrcoef(LN_m, temp[k])[1,0]
        Bonecorr = np.corrcoef(Bone_m, temp[k])[1,0]
        if LNcorr*Bonecorr<0 and abs(LNcorr) >= thresholds[i] and abs(Bonecorr) >= thresholds[i] :
            ggs.append(each)
            idds.append(LNBone_indexs[i][k])
    indexs.append(idds)
    LN_not_Bone.append(np.array(ggs))  # the genes in both LN and Bone, but opposite



plot_gene_LNBone = np.array(LN_not_Bone)
print(plot_gene_LNBone[0].shape)
print(len(indexs[0]))
print(plot_gene_LNBone[1].shape)
print(len(indexs[1]))


3237
2324
(2154, 231)
2154
(1089, 231)
1089


In [65]:
# print(len(LNBone_indexs[0]))
# print(len(LNBone_indexs[1]))
# print(len(rrr[0]))
# print(plot_gene_LNBone[1].shape)
print(len(indexs[0]))
print(len(indexs[1]))
# print(LNindex)
# print(Boneindex)

# union_genes,intersect_genes, difference_genes,sdsdsds,intersect_indexs= gene_selected_by_diff_threshold(norm, thresholds, remove_intersect=True)

# print(intersect_indexs)
print(len(rrr[0]))
print(len(rrr[1]))

# print(len([x for x in indexs[0] if x in intersect_indexs[0]]))
# print(len([x for x in indexs[1] if x in intersect_indexs[1]]))
res = [[x for x in indexs[0] if x in intersect_indexs[0]],[x for x in indexs[1] if x in intersect_indexs[1]]]

2154
1089
559
195


In [102]:
mRNA_231_df=pd.read_csv(mRNA_231_file)
mRNA_231_df = mRNA_231_df.set_index('SAMPLE_ID')
patient_mRNA = []
for column in mRNA_231_df:
    #     print(mRNA_244_df[column].tolist())
    patient_mRNA.append(mRNA_231_df[column].tolist())
mRNA_np = np.asarray(patient_mRNA)

mRNA_np = mRNA_np.T

fitered_genes = []
for i in res[0]:
    fitered_genes.append(mRNA_np[i])
fitered_genes = np.array(fitered_genes)
print(np.array([res[0]]).shape)
print(np.array(fitered_genes).shape)
# fitered_genes = np.append(np.array([res[1]]).reshape(187,1), fitered_genes, axis = 1)
# print(fitered_genes.shape)
print(fitered_genes)
# np.savetxt('C:\\Users\\bird\Desktop\\cancer_project_github\\cancer_data\\filtered_187_genes_data.csv', fitered_genes, delimiter=",")

(1, 524)
(524, 231)
[[8.69300e+00 2.23950e+01 5.08430e+01 ... 2.24079e+02 8.97300e+00
  5.20290e+01]
 [1.16400e+00 3.22020e+01 5.80760e+01 ... 1.12152e+02 5.74000e-01
  1.00740e+01]
 [0.00000e+00 9.18500e+00 0.00000e+00 ... 1.30900e+00 3.16562e+02
  3.13478e+02]
 ...
 [1.60000e-02 4.40000e-02 3.20000e-02 ... 0.00000e+00 1.20000e-02
  1.10000e-01]
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 1.40000e-02 0.00000e+00
  0.00000e+00]
 [6.00000e-03 2.10000e-02 2.50000e-02 ... 2.90000e-02 1.00000e-02
  8.50000e-02]]


In [103]:
sv = pd.DataFrame(fitered_genes,index=res[0])
pd.DataFrame(sv).to_csv('C:\\Users\\bird\Desktop\\cancer_project_github\\cancer_data\\filtered_524_genes_data.csv')

In [28]:
"""the above code plots union genes of three types and selected by both LN and Bone masks, but give opposite correlations.
    In this section, I will plot gene in 'three types' and also 'selected by both LN and Bone masks, 
    but give opposite correlations' give a name to it: intersect.
"""

temp_genes = np.copy(plot_gene_LNBone)
# print(temp_genes[0].shape)
# print(temp_genes[1].shape)

union_genes,intersect_genes, difference_genes,intersect_indexs= gene_selected_by_diff_threshold(norm, thresholds, remove_intersect=True)

print(intersect_indexs)
print(len(intersect_indexs))
plot_genes = intersect_genes
temp2_genes = np.copy(plot_genes)


a = temp2_genes[0,3]
b =  temp2_genes[1,3]
print(a.shape)
print(b.shape)
rres1 =  np.concatenate((a, temp_genes[0]), axis=0)
rres1 = np.unique(rres1, axis=0)
rres2 = np.concatenate((b, temp_genes[1]), axis=0)
rres2 = np.unique(rres2, axis=0)
rres = np.array([rres1,rres2])

# a (559, 231)  three types 0.15
# b (195, 231)   three types 0.20
# temp_genes[0] (2154, 231)  LN_Bone 0.15
# temp_genes[1] (1089, 231)  LN_Bone 0.20
# rres[0](2285, 231)  union 0.15
# rres[1] (1151, 231)  union 0.20


# for 0.15
difference_genes = []
intersect_genes = []
union_rres = set([tuple(x) for x in rres[0]])
three_types = set([tuple(x) for x in a])
LN_Bone = set([tuple(x) for x in temp_genes[0]])
intersect_three_LNBone = [tuple(x) for x in LN_Bone & three_types]
intersect_genes.append(np.array(intersect_three_LNBone))
print(len(intersect_three_LNBone))
intersect_three_LNBone = set(intersect_three_LNBone)


# for 0.20
union_rres = set([tuple(x) for x in rres[1]])
three_types = set([tuple(x) for x in b])
LN_Bone = set([tuple(x) for x in temp_genes[1]])
intersect_three_LNBone = [tuple(x) for x in LN_Bone & three_types]
intersect_genes.append(np.array(intersect_three_LNBone))
print(len(intersect_three_LNBone))
intersect_three_LNBone=set(intersect_three_LNBone)


plot_genes = np.array(intersect_genes)


sorts = {"":'0',
    "homo":'1',
    "hetero":"2"}

norms = {'': '0',
         'linear_scale_and_bin': 'A',
         'log_scale_and_bin': 'A',
         'rank_and_scale': 'B'
}

params = ['homo','mean', '', sorts, norms] # scheme = {'', 'hetero' 'homo'}, norm = {'','rank_and_scale', 'scale_and_bin_linear', 'log_scale_and_bin'}

print(plot_genes.shape)
for i in range(len(plot_genes)):
    patient_np = plot_genes[i].T
#     patient_np = plot_genes[i,3].T   # uncomment this for intersect_genes only

    correlation_matrix, patient_np = norm_and_sort_patients(patient_np, params)#hetero homo scale_and_bin rank_and_scale


    plot_heatmap(correlation_matrix,params)

    

3237
2324
[2, 9, 12, 15, 18, 19, 44, 115, 117, 123, 141, 151, 177, 199, 236, 241, 242, 245, 300, 310, 329, 405, 414, 420, 445, 455, 492, 496, 502, 555, 571, 728, 743, 795, 843, 845, 856, 940, 945, 1031, 1035, 1052, 1055, 1085, 1089, 1122, 1125, 1162, 1196, 1201, 1218, 1237, 1312, 1328, 1440, 1500, 1590, 1594, 1597, 1655, 1814, 1837, 1840, 1860, 1902, 1920, 1932, 1946, 2018, 2081, 2130, 2136, 2162, 2171, 2193, 2206, 2219, 2294, 2369, 2457, 2617, 2815, 2839, 2912, 2997, 3121, 3266, 3312, 3347, 3417, 3419, 3483, 3618, 3705, 3721, 3738, 3928, 3940, 4036, 4062, 4234, 4359, 4377, 4500, 4756, 4774, 4880, 4911, 4932, 5193, 5240, 5415, 5426, 5443, 5498, 5551, 5637, 5742, 5922, 6100, 6271, 6303, 6309, 6416, 6505, 6577, 6599, 6782, 6823, 7000, 7241, 7396, 7397, 7454, 7461, 7531, 7532, 7850, 7863, 7950, 8005, 8075, 8270, 8405, 8718, 8783, 8842, 8916, 8959, 8988, 9152, 9165, 9166, 9184, 9188, 9386, 9414, 9621, 9650, 9748, 10081, 10258, 10430, 10461, 10522, 10557, 10671, 10677, 10819, 10824, 10949, 

ValueError: Image size of 101274x109577 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1440x1440 with 1 Axes>

ValueError: Image size of 101274x109577 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1440x1440 with 1 Axes>